In [ ]:
# create model class
import torch
import torch.nn as nn

In [ ]:
class Model(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features,3),
        nn.ReLU(),
        nn.Linear(3,1),
        nn.Sigmoid(),
    )


  def forward(self, features):
    out = self.network(features)

    return out

  # def loss(self,y)

In [ ]:
# create features
features = torch.rand(10,5)

# create model
model = Model(num_features=features.shape[1])

#call model for forward pass
#model.forward(features)
model(features)


tensor([[0.5400],
        [0.5407],
        [0.5487],
        [0.5365],
        [0.5310],
        [0.5428],
        [0.5402],
        [0.5328],
        [0.5489],
        [0.5431]], grad_fn=<SigmoidBackward0>)

In [ ]:
model.network[0].weight

Parameter containing:
tensor([[-0.0188, -0.3322,  0.1676, -0.1719, -0.0571],
        [-0.4267, -0.0590, -0.2268, -0.2949,  0.4037],
        [ 0.3659, -0.4320,  0.0225, -0.0970,  0.0345]], requires_grad=True)

In [ ]:
model.network[0].bias

Parameter containing:
tensor([ 0.3595, -0.4377,  0.3360], requires_grad=True)

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model,input_size=(10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")

df.drop(columns=["id","Unnamed: 32"], inplace=True)

In [3]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
x_train,x_test,y_train,y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2)

In [5]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [7]:
x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [8]:
from torch.utils.data import Dataset, DataLoader

In [9]:
class CustomDataset(Dataset):
  def __init__(self, features, target):
    self.features = features
    self.target = target

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.target[idx]

In [10]:
train_dataset = CustomDataset(x_train_tensor, y_train_tensor)
test_dataset = CustomDataset(x_test_tensor, y_test_tensor)

In [11]:
train_dataset[10]

(tensor([-0.8950, -0.4052, -0.8694, -0.8338,  0.0090, -0.1620, -0.4722, -0.5539,
         -0.6900,  0.1394, -0.9491,  0.6040, -0.8644, -0.7184,  0.7622,  0.4129,
          0.3561,  0.2532, -0.9163, -0.1241, -0.9656,  0.1176, -0.9202, -0.8554,
          0.4335, -0.0503, -0.3041, -0.2481, -0.9898, -0.0931],
        dtype=torch.float64),
 tensor(0))

In [12]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
import torch.nn as nn

In [26]:
class MySimpleNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()

    # self.linear = nn.Linear(num_features,1)
    # self.sigmoid = nn.Sigmoid()

    self.linear = nn.Sequential(
        nn.Linear(num_features,3),
        nn.ReLU(),
        nn.Linear(3,1),
        nn.Sigmoid()
    )

  def forward(self, features):
      out = self.linear(features)
      # out = self.sigmoid(out)

      return out




In [27]:
learning_rate = 0.1
epochs = 50

In [28]:
loss_function = nn.BCELoss()

In [29]:
model = MySimpleNN(x_train_tensor.shape[1])

#Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):
  for batch_features, batch_labels in train_loader:
    # forward
    y_pred = model(batch_features.float())

    # loss
    loss = loss_function(y_pred, batch_labels.float().view(-1,1))
    # print(f"Epoch: {epoch+1}, loss: {loss.item()}")

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update with no gradient
    optimizer.step()


  print(f"Epoch: {epoch+1}, loss: {loss.item()}")

Epoch: 1, loss: 0.5115707516670227
Epoch: 2, loss: 0.6057049036026001
Epoch: 3, loss: 0.341014564037323
Epoch: 4, loss: 0.31602171063423157
Epoch: 5, loss: 0.7797657251358032
Epoch: 6, loss: 0.2748764157295227
Epoch: 7, loss: 0.334475040435791
Epoch: 8, loss: 0.25052037835121155
Epoch: 9, loss: 0.23565062880516052
Epoch: 10, loss: 0.21380338072776794
Epoch: 11, loss: 0.12068736553192139
Epoch: 12, loss: 0.14514121413230896
Epoch: 13, loss: 0.253284752368927
Epoch: 14, loss: 0.10082205384969711
Epoch: 15, loss: 0.14553584158420563
Epoch: 16, loss: 0.13247957825660706
Epoch: 17, loss: 0.37534263730049133
Epoch: 18, loss: 0.08166055381298065
Epoch: 19, loss: 0.10782003402709961
Epoch: 20, loss: 0.12716057896614075
Epoch: 21, loss: 0.03952890262007713
Epoch: 22, loss: 0.15120208263397217
Epoch: 23, loss: 0.09174934774637222
Epoch: 24, loss: 0.03555305302143097
Epoch: 25, loss: 0.06137116998434067
Epoch: 26, loss: 0.1404331773519516
Epoch: 27, loss: 0.03076629340648651
Epoch: 28, loss: 0.03

In [30]:
# model evaluation
model.eval()
accuracy_list = []
with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    y_pred = model(batch_features.float())
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
    accuracy_list.append(accuracy)
  overall_accuracy = sum(accuracy_list)/len(accuracy_list)
  print(f"Accuracy: {overall_accuracy:.4f}")

Accuracy: 0.9922
